# **1. Install Rclone**

In [ ]:
#@markdown <h3>📝 Note: Run this before using Rclone.</h3>
#@markdown <h4>📝 Upload an rclone profile</4>
Setup_Time_Zone = False 

import os; from google.colab import files; from IPython.display import HTML, clear_output

def upload_conf():
  try:
    display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Please upload the config file of rclone (rclone.conf) from your computer.</h2><br>"))
    UploadConfig = files.upload().keys()
    clear_output(wait=True)
    if len(UploadConfig) == 0:
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
    elif len(UploadConfig) == 1:
      for fn in UploadConfig:
        if os.path.isfile("/content/" + fn) == True:
          os.environ["rclone_conf"] = fn
          !mv -f "$rclone_conf" /root/.rclone.conf
          !chmod 666 /root/.rclone.conf
          if Setup_Time_Zone == True:
            !sudo dpkg-reconfigure tzdata
            clear_output(wait=True)
          if os.path.isfile("/usr/bin/rclone") == True:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">Config has been changed.</h2><br></center>"))
          else:
            !rm -rf /content/sample_data/
            !curl -s https://rclone.org/install.sh | sudo bash
            clear_output(wait=True)
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">Installation has been successfully completed.</h2><br></center>"))
        else:
          return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
    else:
      for fn in UploadConfig:
        os.environ["rclone_conf"] = fn
        !rm -f "$rclone_conf"
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
  except:
    clear_output(wait=True)
    return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))

upload_conf()

**2. Mount OneDrive**   
To stream files we need to mount One Drive.

In [ ]:
# ============================= FORM ============================= #
#@markdown <h3>📝 Run this to mount the drive with rclone</h3>
rclone_config_name = "onedrive" #@param {type:"string"}
local_mount_location = "./onedrive" #@param {type:"string"}
# ============================= FORM ============================= #
import time
import os

# clear nohup
open("nohup.out", 'w').close()


# unmount first

!fusermount -u $local_mount_location 2>/dev/null


# mount without waiting for the command to complete
!mkdir $local_mount_location 2>/dev/null
!nohup rclone mount $rclone_config_name: $local_mount_location --buffer-size 96M & 
  


# Show the output that was written to nohup
time.sleep(3)
f = open(r"nohup.out", "r")
nohupText = f.read()
f.close()


dirs = os.listdir(local_mount_location)

if len(dirs) > 0:
  clear_output(wait=True)
  print("Succeeded. ", str(len(dirs)), "dirs found at", local_mount_location)

else:
  print("\n\nNot succeeded. No files or directories in mounted location. \nCheck your config name and content. If the rclone command was not found, run the cell above.\n\n")
  print("log:\n", nohupText)

**3. Check if mounted correctly**

In [ ]:
!ls ./onedrive

**4. Installing qBitTorrent!**

In [ ]:
# ============================= FORM ============================= #
#@markdown <h3>📝CREATE CONFIG FILE</h3>
connections = 450 
#param {type:"integer"}
savePath = "./onedrive/Media/" #@param {type:"string"}
ngrok_Token = "" #@param {type:"string"}
#https://github.com/thim0o/Google-Colab-Peerflix-Server
# ============================= FORM ============================= #

import os
import logging
import json
import requests
import ast
import threading
import time
import subprocess
import shutil

from IPython import display

#remove existing config file
shutil.rmtree('/root/.config/qBittorrent', ignore_errors=True)

configPath = r"/root/.config/qBittorrent/qBittorrent.conf"


#CREATE THE CONFIG
data = r"""
[AutoRun][AutoRun]
enabled=false
program=chmod -R 666 \"%D\"

[BitTorrent]
Session\CreateTorrentSubfolder=true
Session\DisableAutoTMMByDefault=false
Session\DisableAutoTMMTriggers\CategoryChanged=false
Session\DisableAutoTMMTriggers\CategorySavePathChanged=false
Session\DisableAutoTMMTriggers\DefaultSavePathChanged=true
Session\GlobalMaxSeedingMinutes=1000

[Core]
AutoDeleteAddedTorrentFile=IfAdded

[Preferences]
Downloads\DiskWriteCacheSize=64
Session\SuggestMode=true
Advanced\AnnounceToAllTrackers=true
Session\AnnounceToAllTiers=true
Bittorrent\AddTrackers=true
Bittorrent\MaxConnecs={connections}
Bittorrent\MaxConnecsPerTorrent=-1
Bittorrent\MaxRatio=4
Bittorrent\MaxRatioAction=0
Bittorrent\MaxUploads=8
Bittorrent\PeX=true
Bittorrent\TrackersList=http://nyaa.tracker.wf:7777/announce\n\nhttp://sukebei.tracker.wf:8888/announce\n\nudp://tracker.coppersurfer.tk:6969/announce\n\nudp://tracker.open-internet.nl:6969/announce\n\nudp://tracker.leechers-paradise.org:6969/announce\n\nudp://tracker.internetwarriors.net:1337/announce\n\nudp://tracker.opentrackr.org:1337/announce\n\nudp://9.rarbg.to:2710/announce\n\nudp://9.rarbg.me:2710/announce\n\nhttp://tracker3.itzmx.com:6961/announce\n\nhttp://tracker1.itzmx.com:8080/announce\n\nudp://exodus.desync.com:6969/announce\n\nudp://explodie.org:6969/announce\n\nudp://ipv4.tracker.harry.lu:80/announce\n\nudp://denis.stalker.upeer.me:6969/announce\n\nudp://tracker.torrent.eu.org:451/announce\n\nudp://tracker.tiny-vps.com:6969/announce\n\nudp://thetracker.org:80/announce\n\nudp://open.demonii.si:1337/announce\n\nudp://tracker4.itzmx.com:2710/announce\n\nudp://tracker.cyberia.is:6969/announce\n\nudp://retracker.netbynet.ru:2710/announce\n\nudp://62.138.0.158:6969/announce\n\nudp://188.241.58.209:6969/announce\n\nudp://188.241.58.209:6969/announce\n\nudp://185.225.17.100:1337/announce\n\nudp://62.210.79.110:1337/announce\n\nudp://151.80.120.112:2710/announce\n\nudp://151.80.120.114:2710/announce\n\nhttp://176.113.71.19:6961/announce\n\nhttp://172.64.195.37:8080/announce\n\nudp://208.83.20.20:6969/announce\n\nudp://184.105.151.164:6969/announce\n\nudp://51.15.40.114:80/announce\n\nudp://5.2.79.22:6969/announce\n\nudp://91.216.110.52:451/announce\n\nudp://5.206.58.23:6969/announce\n\nudp://176.31.106.35:80/announce\n\nudp://5.2.79.219:1337/announce\n\nudp://212.1.226.176:2710/announce\n\nudp://159.100.245.181:6969/announce\n\nudp://212.1.226.176:2710/announce\n
Bittorrent\uTP_rate_limited=false
Connection\GlobalDLLimitAlt=0
Connection\GlobalUPLimit=30000
Connection\GlobalUPLimitAlt=0
Downloads\PreAllocation=false
Downloads\SavePath={savePath}
Downloads\ScanDirsV2=@Variant(\0\0\0\x1c\0\0\0\x1\0\0\0\x12\0/\0\x63\0o\0n\0t\0\x65\0n\0t\0/\0\0\0\x2\0\0\0\x1)
Downloads\StartInPause=false
General\Locale=en
General\UseRandomPort=True
Queueing\IgnoreSlowTorrents=true
Queueing\MaxActiveDownloads=8
Queueing\MaxActiveTorrents=15
Queueing\MaxActiveUploads=8
Queueing\QueueingEnabled=true
WebUI\AlternativeUIEnabled=false
WebUI\AuthSubnetWhitelist=@Invalid()
WebUI\AuthSubnetWhitelistEnabled=false
WebUI\ClickjackingProtection=true
WebUI\HTTPS\Enabled=true
WebUI\Port=6006
Downloads\TempPath=/root/Downloads/temp/
Downloads\TempPathEnabled=true
WebUI\HostHeaderValidation=true
WebUI\LocalHostAuth=false
WebUI\Password_ha1=@ByteArray(322517f6a6fc65697957a1f9da700b43)
WebUI\RootFolder=
WebUI\Username=rclonelab


""".format(connections=connections,
          savePath=savePath,
          ).split("\n")


# SAVE THE CONFIG
!mkdir /root/.config/qBittorrent/
open(configPath, 'w').close()

configFile = open(configPath, "a+")

for line in data:
  configFile.write(line + "\n")
  
configFile.close()

# PRINT THE CONFIG FOR DEBUGGING

print("created the config successfully")

f = open(r"nohup.out", "w+").close()

## Mount drive and install qbittorrent
#%cd {savePath}
!add-apt-repository ppa:qbittorrent-team/qbittorrent-stable -y
!apt install qbittorrent-nox

## Installing and authenticating NGROK
print("Installing and authenticating NGROK")

TOKEN=ngrok_Token

def install_ngrok():
    import os
    from zipfile import ZipFile
    from urllib.request import urlretrieve
    
    ngrok64 = 'ngrok-amd64.zip'
    url = 'https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip'
    urlretrieve(url, ngrok64)
    
    with ZipFile(ngrok64, 'r') as zip_ref:
        zip_ref.extractall('/usr/local/bin/')
    os.chmod('/usr/local/bin/ngrok', 0o755)
    os.unlink(ngrok64)

install_ngrok()
if TOKEN != "":
    !ngrok authtoken $TOKEN


## Final execution
def torrent(port):
    command = subprocess.Popen(['qbittorrent-nox', f'--webui-port={port}'])

def ngrok(port):
    
    ngrok_cmd = subprocess.Popen(['ngrok', 'http', str(port)])    
    localhost_url = "http://localhost:4040/api/tunnels"

    time.sleep(1)
    tunnel_url = requests.get(localhost_url).text
    json_data = json.loads(tunnel_url)

    tunnel_url = json_data['tunnels'][0]['public_url']
    # tunnel_url = tunnel_url.replace("https", "http")
    print('Running at localhost: ' + str(port))
    print(tunnel_url)

if __name__ == '__main__':
    
    port = 9999

    thread_torrent = threading.Thread(target = torrent, args=(int(port),))
    thread_ngrok = threading.Thread(target = ngrok, args=(int(port),))

    thread_torrent.start()
    
    #Clear cell output
    display.clear_output(wait=True)

    print('Torrent server started!')

    time.sleep(5)
    print('Establishing secure connection!')
    
    thread_ngrok.start()
    print('Secure connection established...')
    #print('Username: rclonelab')
    #print('password: @ByteArray(322517f6a6fc65697957a1f9da700b43)')
    
    thread_ngrok.join()
    
    thread_torrent.join()

#remove .out file
myDir= os.listdir(savePath)

for item in myDir:
    if item.endswith(".out"):
        os.remove(os.path.join(savePath, item))


`rclone_config_name` same as rclone `config name`  
`local_mount_location` no need to change  
`savePath` set as per your requirement
`ngrok_Token` get from https://ngrok.com/